# FAMIC Model Tutorial (Google Colab)

This notebook demonstrates how to:
1. Load datasets from HuggingFace
2. Load tokenizers for text preprocessing
3. Initialize the FAMIC model architecture
4. Load pretrained weights and evaluate the model

## Setup for Google Colab

This notebook is designed to run in Google Colab. It will automatically clone the repository and set up the environment.


In [ ]:
# Clone the repository and change directory
import os
from pathlib import Path

# Repository URL
REPO_URL = "https://github.com/YCY198888/FAMIC.git"
REPO_NAME = "FAMIC"

# Clone the repository if it doesn't exist
if not os.path.exists(REPO_NAME):
    print(f"Cloning repository from {REPO_URL}...")
    os.system(f"git clone {REPO_URL}")
    print("✓ Repository cloned successfully!")
else:
    print(f"✓ Repository already exists at {REPO_NAME}")

# Change to the repository directory
os.chdir(REPO_NAME)
project_root = Path().resolve()

print(f"\n✓ Changed to repository directory: {project_root}")
print(f"Current working directory: {os.getcwd()}")


In [ ]:
# Install dependencies
print("Installing dependencies from requirements.txt...")
%pip install -q -r requirements.txt
print("✓ Dependencies installed successfully!")


In [ ]:
# Add src to path
import sys
from pathlib import Path

# Get the project root directory (current directory after cd)
project_root = Path().resolve()
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")
print(f"Python path updated")


In [ ]:
# Import necessary libraries
import torch
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from pathlib import Path

# Import FAMIC modules
from src.datasets import (
    load_dataset_csv,
    print_dataset_statistics,
    report_dataset_statistics,
    load_tokenizer,
    get_dataset_info,
    DATASET_REGISTRY
)

from src.model import (
    FAMIC,
    create_embedding_matrix,
    initialize_model_blocks,
    EMBEDDING_DIMENSIONS,
    VOCAB_LENGTH,
    MAX_LEN,
    NUM_HEADS
)

from src.download_weights import (
    download_all_weights,
    load_pretrained_weights,
    get_weights_path
)

print("✓ All imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")


### Clearing Cached Data

If you need to re-download datasets (e.g., after corrections on HuggingFace), you can clear the cache:


In [ ]:
# Clear cached dataset to force re-download
# Uncomment the line below to clear cache for a specific dataset

from src.datasets import clear_dataset_cache, clear_all_cache

# Clear only the dataset cache
#clear_dataset_cache("twitter")

# Or clear both dataset and tokenizer cache
# clear_all_cache("twitter")

# Or clear everything including HuggingFace internal cache
# clear_all_cache("twitter", clear_hf_cache=True)

print("Cache clearing functions are available.")
print("Uncomment the lines above to clear cache when needed.")


## HuggingFace Authentication

**Important**: If the repository is private or gated, you need to authenticate with HuggingFace first.

You have three options:

1. **Set environment variable** (recommended):
   ```python
   import os
   os.environ['HF_TOKEN'] = 'your_huggingface_token_here'
   ```

2. **Use the login function**:
   ```python
   from src.datasets import authenticate_huggingface
   authenticate_huggingface(token='your_huggingface_token_here')
   ```

3. **Use HuggingFace CLI** (before starting Jupyter):
   ```bash
   huggingface-cli login
   ```

Get your token from: https://huggingface.co/settings/tokens

**Note**: If you're not sure if authentication is needed, try loading the dataset first. The error message will tell you if authentication is required.


In [ ]:
# Optional: Authenticate with HuggingFace if needed
# Uncomment and add your token if you get authentication errors

# Option 1: Set environment variable
# import os
# os.environ['HF_TOKEN'] = 'your_huggingface_token_here'

# Option 2: Use login function
# from src.datasets import authenticate_huggingface
# authenticate_huggingface(token='your_huggingface_token_here')

# Option 3: If you've already logged in via CLI, you can skip this cell
print("Authentication: If you get 401 errors, uncomment one of the options above")


## 1. Loading Datasets

The FAMIC model supports two datasets:
- **Twitter**: Twitter dataset cleaned in 2024
- **Wine**: Wine dataset with 140k samples cleaned in 2025

Both datasets are automatically downloaded from HuggingFace on first use and cached locally.


In [ ]:
# Show available datasets
print("Available datasets:")
for name, info in DATASET_REGISTRY.items():
    print(f"  - {name}: {info['name']}")
    print(f"    Description: {info['description']}")
    print()


In [ ]:
# Choose which dataset to load
# Options: "twitter" or "wine"
DATASET_NAME = "wine"  # Change this to "wine" to load the wine dataset

print(f"Loading {DATASET_NAME} dataset...")
print("="*70)

# Load the dataset
df = load_dataset_csv(DATASET_NAME)

# Display basic information
print(f"\nDataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"\nFirst few rows:")
df.head()


In [ ]:
# Get detailed dataset statistics
print_dataset_statistics(DATASET_NAME, df=df)


## 2. Loading Tokenizers

Each dataset has its own tokenizer that was used during training. The tokenizers are also downloaded from HuggingFace.


In [ ]:
# Load the tokenizer for the selected dataset
print(f"Loading tokenizer for '{DATASET_NAME}' dataset...")
print("="*70)

tokenizer = load_tokenizer(DATASET_NAME)

# Display tokenizer information
print(f"\nTokenizer loaded successfully!")
print(f"Vocabulary size: {len(tokenizer.word_index) if hasattr(tokenizer, 'word_index') else 'N/A'}")
print(f"OOV token: {tokenizer.oov_token if hasattr(tokenizer, 'oov_token') else 'N/A'}")


In [ ]:
# Example: Tokenize some sample text
if hasattr(df, 'columns'):
    # Try to find a text column (common names)
    text_columns = [col for col in df.columns if any(keyword in col.lower() 
                   for keyword in ['text', 'review', 'comment', 'content', 'tweet'])]
    
    if text_columns:
        sample_text = df[text_columns[0]].iloc[0] if len(df) > 0 else "This is a sample text."
        print(f"Sample text: {sample_text}")
        print(f"\nTokenizing sample text...")
        
        # Tokenize
        sequences = tokenizer.texts_to_sequences([sample_text])
        print(f"Tokenized sequence: {sequences[0]}")
        print(f"Sequence length: {len(sequences[0])}")
    else:
        print("No text column found. Please adjust the column name in the code above.")
        print(f"Available columns: {list(df.columns)}")
else:
    print("DataFrame not available")


## 3. Initializing the FAMIC Model

Now we'll initialize the FAMIC model architecture. The model requires:
- An embedding matrix (can be empty or pre-trained)
- Model hyperparameters

### 3.1 Create Embedding Matrix

First, we create an embedding matrix. For now, we'll create an empty one that can be filled with pre-trained embeddings later.


In [ ]:
# Create embedding matrix
# In practice, you would load pre-trained embeddings (e.g., Word2Vec, GloVe)
# For now, we create an empty matrix that matches the expected dimensions

embedding_dim = EMBEDDING_DIMENSIONS  # 300
vocab_length = VOCAB_LENGTH  # 250000

print(f"Creating embedding matrix...")
print(f"  Vocabulary size: {vocab_length + 1}")
print(f"  Embedding dimension: {embedding_dim}")

embedding_matrix = create_embedding_matrix(
    vocab_length=vocab_length,
    embedding_dim=embedding_dim
)

print(f"\n✓ Embedding matrix created")
print(f"  Shape: {embedding_matrix.shape}")
print(f"  Dtype: {embedding_matrix.dtype}")


### 3.2 Initialize Model Blocks

We can initialize individual model blocks or the complete FAMIC model.


In [ ]:
# Initialize individual model blocks
print("Initializing FAMIC model blocks...")
print("="*70)

blocks = initialize_model_blocks(
    embedding_matrix=embedding_matrix,
    hidden_dim=EMBEDDING_DIMENSIONS,
    n_layers=2,
    max_relative_position_mask=2,
    max_relative_position_shift=5,
    pivot=0.5,
    num_heads=NUM_HEADS,
    drop_prob=0.5,
    digits_dim=1
)

print("\n✓ Model blocks initialized:")
for block_name in blocks.keys():
    print(f"  - {block_name}: {type(blocks[block_name]).__name__}")


In [ ]:
# Initialize the complete FAMIC model
print("Initializing complete FAMIC model...")
print("="*70)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = FAMIC(
    embedding_matrix=embedding_matrix,
    hidden_dim=EMBEDDING_DIMENSIONS,
    n_layers=2,
    max_relative_position_mask=2,
    max_relative_position_shift=5,
    pivot=0.5,
    num_heads=NUM_HEADS,
    drop_prob=0.5,
    digits_dim=1
)

model = model.to(device)
model.eval()  # Set to evaluation mode

print("\n✓ FAMIC model initialized successfully!")
print(f"  Model device: {next(model.parameters()).device}")


### 3.3 Load Pretrained Weights from HuggingFace

Now we can load pretrained weights for the selected dataset. The weights are automatically downloaded from HuggingFace if not already cached.


In [ ]:
# Load pretrained weights from HuggingFace
# This will automatically download weights if not already cached
print("Loading pretrained weights from HuggingFace...")
print("="*70)

from src.download_weights import load_pretrained_weights

# Load weights into the model blocks
model_blocks_loaded = load_pretrained_weights(
    model_blocks={
        'embeds': model.embeds,
        'sentiment': model.sentiment,
        'mask': model.mask,
        'shifter1': model.shifter1,
        'shifter2': model.shifter2,
        'synthesizer': model.synthesizer
    },
    dataset_name=DATASET_NAME,
    cache_dir="models",
    version="v2",  # Model version
    device=device
)

print("\n✓ Pretrained weights loaded successfully!")
print(f"  Model is ready for inference on '{DATASET_NAME}' dataset")


**Alternative: Initialize model with pretrained weights directly**

You can also create a model and load weights in one step:


In [ ]:
# Alternative: Create model with pretrained weights in one step
print("Alternative method: Loading model with pretrained weights...")
print("="*70)

# Create a new model instance with pretrained weights
model_pretrained = FAMIC.from_pretrained_huggingface(
    dataset_name=DATASET_NAME,
    embedding_matrix=embedding_matrix,
    cache_dir="models",
    version="v2",
    device=device,
    hidden_dim=EMBEDDING_DIMENSIONS,
    n_layers=2,
    max_relative_position_mask=2,
    max_relative_position_shift=5,
    pivot=0.5,
    num_heads=NUM_HEADS,
    drop_prob=0.5,
    digits_dim=1
)

print("\n✓ Model with pretrained weights created successfully!")
print(f"  Model device: {next(model_pretrained.parameters()).device}")


## 4. Creating Train/Validation/Test Splits

For evaluation, we need to split the dataset into train, validation, and test sets. The model is trained on the training set, early stopped using the validation set, and finally evaluated on the test set.


**Note**: If you get an `ImportError` for `WordDataset`, restart your kernel or run the cell below to reload the module.


In [ ]:
# Force reload the datasets module to pick up new classes
import importlib
import sys

# Remove the module from cache if it exists
if 'src.datasets' in sys.modules:
    del sys.modules['src.datasets']

# Re-import
from src.datasets import (
    WordDataset,
    create_train_val_test_split,
    create_dataloaders,
    load_dataset_csv,
    load_tokenizer
)

print("✓ Module reloaded successfully")
print("✓ WordDataset and other functions are now available")


In [ ]:
# Create train/validation/test splits and dataloaders
from src.datasets import create_dataloaders, create_train_val_test_split

# Configuration
MAX_LEN = 150  # Maximum sequence length
BATCH_SIZE = 100  # Batch size for training/evaluation
SEED = 2025  # Random seed for reproducibility

# Column names (adjust these based on your dataset structure)
# Common names: 'text', 'preprocessed_text', 'review', 'tweet', etc.
TEXT_COLUMN = "preprocessed_text"  # Adjust if your dataset uses a different column name
LABEL_COLUMN = "labels"  # Adjust if your dataset uses a different column name

print(f"Creating dataloaders for '{DATASET_NAME}' dataset...")
print("="*70)

# Create dataloaders
train_loader, val_loader, test_loader = create_dataloaders(
    dataset_name=DATASET_NAME,
    text_column=TEXT_COLUMN,
    label_column=LABEL_COLUMN,
    max_len=MAX_LEN,
    batch_size=BATCH_SIZE,
    test_size=0.1,  # 10% for test
    val_size=0.5,   # 50% of test set for validation (so 5% total)
    random_state=SEED,
    num_workers=0
)


In [ ]:
# Verify the dataloaders work
print("\nTesting dataloaders...")
print("="*70)

# Get a sample batch from train loader
sample_batch = next(iter(train_loader))
print(f"\nSample batch from train loader:")
print(f"  word_ids shape: {sample_batch['word_ids'].shape}")
print(f"  attention_mask shape: {sample_batch['attention_mask'].shape}")
print(f"  labels shape: {sample_batch['labels'].shape}")
print(f"  Batch size: {sample_batch['word_ids'].shape[0]}")
print(f"  Sequence length: {sample_batch['word_ids'].shape[1]}")

# Show a sample
print(f"\nSample from batch:")
print(f"  First word_ids (first 20 tokens): {sample_batch['word_ids'][0][:20].tolist()}")
print(f"  First attention_mask (first 20): {sample_batch['attention_mask'][0][:20].tolist()}")
print(f"  First label: {sample_batch['labels'][0].item()}")


## 5. Evaluating the Model

Now that we have the model, pretrained weights, and test dataloader, we can evaluate the model performance on the test set.


In [ ]:
# Evaluate the model on test set
from src.evaluate import evaluate_model
import torch.nn as nn

# Set evaluation flags (these should match how the model was trained)
USE_MASK = True
USE_SHIFT1 = True
USE_SHIFT2 = True

# Use the model with pretrained weights (model_pretrained from earlier)
# Or use the model after loading weights (model from earlier)

print("Evaluating model on test set...")
print("="*70)

# Define loss function (BCEWithLogitsLoss for binary classification)
criterion = nn.BCEWithLogitsLoss()

# Evaluate on test set
test_metrics = evaluate_model(
    model=model_pretrained,  # Use the model with pretrained weights
    dataloader=test_loader,
    device=device,
    criterion=criterion,
    use_mask=USE_MASK,
    use_shift1=USE_SHIFT1,
    use_shift2=USE_SHIFT2,
    return_loss=True
)

print("\n✓ Evaluation complete!")
print(f"Test Accuracy: {test_metrics['accuracy']:.4f}")
print(f"Test F1-Score: {test_metrics['f1_score']:.4f}")


### 5.1 Detailed Evaluation Metrics

Let's examine the detailed evaluation results:


In [ ]:
# Display detailed evaluation metrics
print("="*70)
print("DETAILED EVALUATION METRICS")
print("="*70)
print(f"\nDataset: {DATASET_NAME}")
print(f"Evaluation Configuration:")
print(f"  Use Mask: {USE_MASK}")
print(f"  Use Shifter 1: {USE_SHIFT1}")
print(f"  Use Shifter 2: {USE_SHIFT2}")
print(f"\nPerformance Metrics:")
print(f"  Accuracy:  {test_metrics['accuracy']:.4f}")
print(f"  Precision: {test_metrics['precision']:.4f}")
print(f"  Recall:    {test_metrics['recall']:.4f}")
print(f"  F1-Score:  {test_metrics['f1_score']:.4f}")
if 'loss' in test_metrics:
    print(f"  Loss:      {test_metrics['loss']:.4f}")

print(f"\nConfusion Matrix Breakdown:")
print(f"  True Positives (TP):  {test_metrics['TP']}")
print(f"  True Negatives (TN):  {test_metrics['TN']}")
print(f"  False Positives (FP): {test_metrics['FP']}")
print(f"  False Negatives (FN): {test_metrics['FN']}")
print("="*70)


In [ ]:
# Generate classification report
from sklearn.metrics import classification_report

# Get predictions and true labels from the metrics
# Note: We need to reconstruct these from the confusion matrix or re-run evaluation
# For now, let's create a simple report based on the confusion matrix

print("Classification Report:")
print("="*70)

# Calculate per-class metrics from confusion matrix
cm = test_metrics['confusion_matrix']
TN, FP, FN, TP = cm[0, 0], cm[0, 1], cm[1, 0], cm[1, 1]

# Calculate per-class precision, recall, F1
precision_class0 = TN / (TN + FN + 1e-12)  # Precision for class 0
recall_class0 = TN / (TN + FP + 1e-12)     # Recall for class 0
f1_class0 = 2 * precision_class0 * recall_class0 / (precision_class0 + recall_class0 + 1e-12)

precision_class1 = TP / (TP + FP + 1e-12)  # Precision for class 1
recall_class1 = TP / (TP + FN + 1e-12)     # Recall for class 1
f1_class1 = 2 * precision_class1 * recall_class1 / (precision_class1 + recall_class1 + 1e-12)

print(f"\nClass 0 (Negative):")
print(f"  Precision: {precision_class0:.4f}")
print(f"  Recall:    {recall_class0:.4f}")
print(f"  F1-Score:  {f1_class0:.4f}")
print(f"  Support:   {TN + FP}")

print(f"\nClass 1 (Positive):")
print(f"  Precision: {precision_class1:.4f}")
print(f"  Recall:    {recall_class1:.4f}")
print(f"  F1-Score:  {f1_class1:.4f}")
print(f"  Support:   {TP + FN}")

print(f"\nOverall:")
print(f"  Accuracy:  {test_metrics['accuracy']:.4f}")
print(f"  Total:     {TN + FP + TP + FN}")
print("="*70)


### 5.2 Evaluation Summary

Summary of the evaluation results:


In [ ]:
# Create a summary dictionary for easy access
evaluation_summary = {
    'dataset': DATASET_NAME,
    'configuration': {
        'use_mask': USE_MASK,
        'use_shift1': USE_SHIFT1,
        'use_shift2': USE_SHIFT2
    },
    'metrics': {
        'accuracy': test_metrics['accuracy'],
        'precision': test_metrics['precision'],
        'recall': test_metrics['recall'],
        'f1_score': test_metrics['f1_score']
    },
    'confusion_matrix': {
        'TP': test_metrics['TP'],
        'TN': test_metrics['TN'],
        'FP': test_metrics['FP'],
        'FN': test_metrics['FN']
    }
}

if 'loss' in test_metrics:
    evaluation_summary['metrics']['loss'] = test_metrics['loss']

print("Evaluation Summary:")
print("="*70)
print(f"Dataset: {evaluation_summary['dataset']}")
print(f"\nConfiguration:")
for key, value in evaluation_summary['configuration'].items():
    print(f"  {key}: {value}")
print(f"\nMetrics:")
for key, value in evaluation_summary['metrics'].items():
    print(f"  {key}: {value:.4f}")
print(f"\nConfusion Matrix:")
for key, value in evaluation_summary['confusion_matrix'].items():
    print(f"  {key}: {value}")
print("="*70)

# Store summary for later use
print("\n✓ Evaluation summary stored in 'evaluation_summary' variable")


In [ ]:
# Visualize confusion matrix
from src.evaluate import plot_confusion_matrix
import os

# Create results directory if it doesn't exist
os.makedirs("results", exist_ok=True)

plot_confusion_matrix(
    confusion_matrix=test_metrics['confusion_matrix'],
    class_names=["Negative", "Positive"],  # Adjust based on your dataset
    save_path="results/test_confusion_matrix.png",
    title=f"Test Set Confusion Matrix - {DATASET_NAME}"
)


## Summary

This notebook demonstrated:

1. ✅ **Cloning the repository** and setting up the environment in Google Colab
2. ✅ **Loading datasets** from HuggingFace with automatic caching
3. ✅ **Loading tokenizers** for text preprocessing
4. ✅ **Initializing the FAMIC model** with all its components
5. ✅ **Loading pretrained weights** from HuggingFace for Twitter and Wine datasets
6. ✅ **Creating train/validation/test splits** with reproducible random seed (2025)
7. ✅ **Creating PyTorch DataLoaders** for model training and evaluation
8. ✅ **Evaluating the model** on the test set with comprehensive metrics

### Next Steps

- Use the model for inference on new data
- Experiment with different datasets (twitter/wine)
- Fine-tune the model on your own data
- Explore the model's interpretability features

### Notes

- Datasets are cached in the `data/` directory after first download
- Tokenizers are cached in the `data/tokenizers/` directory
- Model weights are cached in the `models/{dataset_name}/` directory
- Data splits use random seed 2025 for reproducibility
- Default split: 90% train, 5% validation, 5% test
- Make sure to use the correct tokenizer and weights for each dataset
- Weights are automatically downloaded from HuggingFace on first use
- In Colab, files are saved to the `/content/FAMIC/` directory
